In [1]:
import sys
import timeit
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.core      as coti

import pyoti.real      as r
import pyoti.sparse    as oti

# Static libraries:
# Dual
import pyoti.static.onumm1n1  as dual

# Multiduals
import pyoti.static.mdnum2    as md2
# import pyoti.static.mdnum3    as md3
# import pyoti.static.mdnum4    as md4
# import pyoti.static.mdnum5    as md5

# OTI.
import pyoti.static.onumm1n2  as om1n2 
# import pyoti.static.onumm1n3  as om1n3 
# import pyoti.static.onumm1n5  as om1n5 
# import pyoti.static.onumm2n2  as om2n2 
# import pyoti.static.onumm3n3  as om3n3 
# import pyoti.static.onumm1n10 as om1n10
# import pyoti.static.onumm10n1 as om10n1
# import pyoti.static.onumm2n5  as om2n5 

np = oti.np


# modules = [r, dual, md2, md3, md4, md5, om1n2, om1n3, om1n5, om2n2, om3n3, om1n10, om10n1, om2n5]
modules = [r, dual, md2, om1n2]

In [2]:
x = md2.zero(nip=3)

In [3]:
x[2]=3.5

In [11]:
x

mdnumfe2< nip: 3, 
(0) 0.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([1,2])
(1) 0.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([1,2])
(2) 3.5000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([1,2])
>

In [31]:
report_length = 50
def report_times():
    for i in range(len(modules)):
        mname = modules[i].__name__
        print(mname+" ::::")
        for key1, items1 in times[i].items():
            for key2, items2 in items1.items():
                str_out = key1+" "+key2 + ' '
                str_out += '-'*(int(report_length-len(str_out))) 
                print(str_out+" {0:15.4f} ns".format((items2)) )
        print()
        print()
        print()
        
def report_times_comp_real():
    mreal = modules[0]
    treal = times[0]
    for i in range(1,len(modules)):
        mname = modules[i].__name__
        print(mname+" ::::")
        for key1, items1 in times[i].items():
            for key2, items2 in items1.items():
                item_real = treal[key1][key2]
                str_out = key1+" "+key2 + ' '
                str_out += '-'*(int(report_length-len(str_out))) 
                print(str_out+" {0:15.2f}X".format((items2/item_real)) )
        print()
        print()
        print()

In [5]:
times = []
niter = int(1e5)
for module in modules:
    mat_size = 18
    nip      = 9
    print("Analyzing "+module.__name__ + str((mat_size,mat_size)))
    times.append({})
    times[-1]['scalar']={}
    times[-1]['gauss_scalar']={}
    times[-1]['array']={}
    times[-1]['gauss_array']={}

    niter = int(1e5)
    
    x  = module.number(0.5)
    try:
        xout = x.copy()
    except:
        xout=x
    # end try
    
    # Create the objects.
    X  = module.zeros((mat_size,mat_size))
    Xout = X.copy()
    xg = module.number(0.5,  nip=nip)
    xgout = xg.copy()
    Xg = module.zeros((mat_size,mat_size), nip=nip)
    Xgout = Xg.copy()

    expr = 'x+x'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'x*x'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(x)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.pow(x,100.)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'X+X'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'X*X'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.dot(X,X)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(X)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sum(x,x,out=xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.mul(x,x,out=xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(x, out=xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.pow(x,100., out=xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sum(X,X,out=Xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.mul(X,X,out=Xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.dot(X,X,out=Xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(X,out=Xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    
    
    
    
    niter = int(1e4)
    expr = 'xg+xg'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'xg*xg'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(xg)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.gauss_integrate(xg,xg)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    
    
    expr = 'Xg+Xg'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'Xg*Xg'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.dot(Xg,Xg)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(Xg)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.gauss_integrate(Xg,xg)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    
    
    
    expr = 'module.sum(xg,xg,out=xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.mul(xg,xg,out=xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(xg,out=xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.gauss_integrate(xg,xg,out=x)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['scalar'][expr.replace('module.','')]=time/float(niter)*1e9
    
    
    
    expr = 'module.sum(Xg,Xg,out=Xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.mul(Xg,Xg,out=Xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.dot(Xg,Xg,out=Xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.sin(Xg,out=Xgout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
    
    expr = 'module.gauss_integrate(Xg,xg,out=Xout)'
    time = timeit.timeit (expr,globals=globals(), number =niter)
    times[-1]['gauss_array'][expr.replace('module.','')]=time/float(niter)*1e9
# end for 
print("Finished")


Analyzing pyoti.real(18, 18)
Analyzing pyoti.onumm1n1(18, 18)
Analyzing pyoti.mdnum2(18, 18)
Analyzing pyoti.mdnum3(18, 18)


KeyboardInterrupt: 

In [23]:
report_times()

pyoti.real ::::
scalar x+x -----------------------------         46.9002 ns
scalar x*x -----------------------------         47.9003 ns
scalar sin(x) --------------------------      34981.6216 ns
scalar pow(x,100.) ---------------------      35109.6456 ns
scalar xg+xg ---------------------------        214.1845 ns
scalar xg*xg ---------------------------        200.8744 ns
scalar sin(xg) -------------------------        412.4792 ns
scalar gauss_integrate(xg,xg) ----------        158.3919 ns
array X+X ------------------------------        215.4491 ns
array X*X ------------------------------        221.6272 ns
array dot(X,X) -------------------------        343.9888 ns
array sin(X) ---------------------------        295.4683 ns
gauss_array Xg+Xg ----------------------       8046.4698 ns
gauss_array Xg*Xg ----------------------       7544.1003 ns
gauss_array dot(Xg,Xg) -----------------       8764.8825 ns
gauss_array sin(Xg) --------------------       8106.0519 ns
gauss_array gauss_integr

In [32]:
report_times_comp_real()

pyoti.onumm1n1 ::::
scalar x+x ---------------------------------------            1.34X
scalar x*x ---------------------------------------            1.57X
scalar sin(x) ------------------------------------            0.00X
scalar pow(x,100.) -------------------------------            0.01X
scalar sum(x,x,out=xout) -------------------------            0.77X
scalar mul(x,x,out=xout) -------------------------            0.83X
scalar sin(x, out=xout) --------------------------            0.01X
scalar pow(x,100., out=xout) ---------------------            0.01X
scalar xg+xg -------------------------------------            2.50X
scalar xg*xg -------------------------------------            2.71X
scalar sin(xg) -----------------------------------            1.90X
scalar gauss_integrate(xg,xg) --------------------            2.02X
scalar sum(xg,xg,out=xgout) ----------------------            1.04X
scalar mul(xg,xg,out=xgout) ----------------------            1.50X
scalar sin(xg,out=xgout) ---

In [12]:
# Test Scalar

module = md2

x = 0.5+module.e(1)+module.e(2)
print(module.__name__+" - Scalar")
print("(Scalar) Addition")
%timeit x+x
print("\n(Scalar) Multiplication")
%timeit x*x
print("\n(Scalar) Function - sine")
%timeit module.sin(x)
print("\n(Scalar) Function - x**100")
%timeit module.pow(x,100.)

pyoti.mdnum2 - Scalar
(Scalar) Addition
81.2 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

(Scalar) Multiplication
87.3 ns ± 5.94 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

(Scalar) Function - sine
254 ns ± 67.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Scalar) Function - x**100
227 ns ± 10 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
# Test Array
X = module.zeros((3,3)) + x
print(module.__name__+" - Array "+str(X.shape))
print("(Array) Addition")
%timeit X+X
print("\n(Array) Multiplication (elementwise)")
%timeit X*X
print("\n(Array) Multiplication (elementwise)")
%timeit module.dot(X,X)
print("\n(Array) Function - sine")
%timeit module.sin(X)
print("\n(Array) Function - x**100")
%timeit module.pow(X,100.)

pyoti.mdnum2 - Array (3, 3)
(Array) Addition
271 ns ± 7.09 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Array) Multiplication (elementwise)
324 ns ± 54.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Array) Multiplication (elementwise)
624 ns ± 32.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Array) Function - sine
722 ns ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Array) Function - x**100
952 ns ± 83.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
# Test Gauss scalar
x = module.number(0.5,nip=27)+module.e(1)+module.e(2)
print(module.__name__+" - Gauss Scalar with "+str(x.nip)+" integration points")
print("(Gauss Scalar) Addition")
%timeit x+x
print("\n(Gauss Scalar) Multiplication")
%timeit x*x
print("\n(Gauss Scalar) Function - sine")
%timeit module.sin(x)
print("\n(Gauss Scalar) Function - x**100")
%timeit module.pow(x,100.)
print("\n(Gauss Scalar) Gauss integration")
%timeit module.gauss_integrate(x,x)

pyoti.mdnum2 - Gauss Scalar with 27 integration points
(Gauss Scalar) Addition
302 ns ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Gauss Scalar) Multiplication
344 ns ± 5.66 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Gauss Scalar) Function - sine
1.01 µs ± 26.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Gauss Scalar) Function - x**100
1.54 µs ± 21.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

(Gauss Scalar) Gauss integration
288 ns ± 5.18 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
# Test Gauss Array
X = module.zeros((3,3), nip=27) + 0.5 + module.e(1)
print(module.__name__+" - Gauss Array "+str(X.shape))
print("(Gauss Array) Addition")
%timeit X+X
print("\n(Gauss Array) Multiplication (elementwise)")
%timeit X*X
print("\n(Gauss Array) Multiplication (elementwise)")
%timeit module.dot(X,X)
print("\n(Gauss Array) Function - sine")
%timeit module.sin(X)
print("\n(Gauss Array) Function - x**100")
%timeit module.pow(X,100.)
print("\n(Gauss Array) Gauss integration")
%timeit module.gauss_integrate(X,x)

pyoti.mdnum2 - Gauss Array (3, 3)
(Gauss Array) Addition
8.61 µs ± 563 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

(Gauss Array) Multiplication (elementwise)
8.76 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

(Gauss Array) Multiplication (elementwise)
12.8 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)

(Gauss Array) Function - sine
15.6 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)

(Gauss Array) Function - x**100
20.1 µs ± 966 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

(Gauss Array) Gauss integration
1.51 µs ± 69.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
globals()['md3']

<module 'pyoti.mdnum3' from '../../../build/pyoti/mdnum3.cpython-37m-darwin.so'>

In [21]:
f = oti.sin(x)

In [23]:
f.get_deriv([1,2,3])

-5.2654953713422366

In [30]:
X_so  = oti.zeros((3,3)) +1+ oti.e(1)+ oti.e(2)+ oti.e(3)+ oti.e(4)+ oti.e(5)+ oti.e(6)+ oti.e(7)+ oti.e(8)+ oti.e(9)+ oti.e(10)
%timeit oti.dot(X_so,X_so)

8.13 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
X_om10n1 = om10n1.zeros((3,3))
%timeit om10n1.dot(X_om10n1,X_om10n1)

1.08 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
X_om2n5 = om2n5.zeros((3,3))
out = X_om2n5.copy()
%timeit om2n5.dot(X_om2n5,X_om2n5,out=out)

3.88 µs ± 466 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [34]:
X_so  = oti.zeros((3,3)) +(1 + oti.e(1,order=5)+ oti.e(2,order=5))**5
out = X_so.copy()
%timeit oti.dot(X_so,X_so,out=out)

53 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
X_om2n5 = om2n5.zeros((3,3))

X_om2n5

omatm2n5< shape: (3, 3), 
 - Column 0
(0,0) 0.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([[1,2]]) + 0.0000 * e([1,2]) + 0.0000 * e([[2,2]]) + nan * e([[1,3]]) + 0.0000 * e([[1,2],2]) + 109334514378330524926448821833027509667024039306198117085945031628731651357239362913973692614628700194714064106577882965190085487003873865419878680597402923827307140804237866399442308069567322368397051102734317435256912538833217182449182393619012647684931584.0000 * e([1,[2,2]]) + 0.0000 * e([[2,3]]) + 0.0000 * e([[1,4]]) + 0.0000 * e([[1,3],2]) + 0.0000 * e([[1,2],[2,2]]) + nan * e([1,[2,3]]) + 0.0000 * e([[2,4]]) + nan * e([[1,5]]) + 0.0000 * e([[1,4],2]) + 11711902778936637226412060440200741170897204695545250784611653264892894013472031834471901470953311875452542224830134818818813432582736715921688651224358315997421109248.0000 * e([[1,3],[2,2]]) + 0.0000 * e([[1,2],[2,3]]) + 0.0000 * e([1,[2,4]]) + 0.0000 * e([[2,5]])
(1,0) 0.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([[1,2]]) + 0.

In [21]:
X_np = np.zeros((3,3))

In [22]:
%timeit np.dot(X_np,X_np)

1.38 µs ± 7.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
X_r = r.zeros((3,3))

In [20]:
%timeit r.dot(X_r,X_r)

352 ns ± 2.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
base = {}
base[r]=r
base[dual]=dual
base[md2]=md2
base[md3]=md3
base[md4]=md4
base[md5]=md5
base[om1n2]=om1n2
base[om1n3]=om1n3
base[om1n5]=om1n5
base[om1n10]=om1n10
base[om10n1]=om10n1
base[om2n5]=om2n5
base[om2n2]=om2n2
base[om3n3]=om3n3
#om1n2, om1n3, om1n5, om2n2, om3n3, om1n10, om10n1, om2n5

In [10]:
%timeit base[om3n3]

58.8 ns ± 0.933 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
x = 0.95+om1n2.e(1)
om1n2.sin(x**2)

0.7849 + 1.1773 * e([1]) - 0.7971 * e([[1,2]])

In [8]:
200*199*.95**198

1.5458128716934985

In [9]:
(x**200).get_deriv([[1,2]])

1.5458128716934985